In [7]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import os, shutil
import zipfile

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

img_rows = 150
img_cols = 150




In [8]:
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip") as train:
    train.extractall(".")
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip") as test:
    test.extractall(".")    

In [9]:
import matplotlib.pyplot as plt  
    
def PlotTwoAccuracyComparison(acc1, val_acc1, acc2, val_acc2, lab1 = 'Model 1', lab2 = 'Model 2'):
    plt.clf()   
    
    plt.rcParams['figure.figsize'] = (25.0, 5.0) 
    epochs = range(len(acc1))
    
    plt.plot(epochs, acc1, 'bo', label='Training accuracy for ' + lab1)
    plt.plot(epochs, val_acc1, 'b', label='Validation accuracy for ' + lab1)
    plt.plot(epochs, acc2, 'ro', label='Training accuracy for ' + lab2)
    plt.plot(epochs, val_acc2, 'r', label='Validation accuracy for ' + lab2)
    plt.title('Comparison of Training and Validation Accuracies')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
def PlotTwoLossComparison(loss1, val_loss1, loss2, val_loss2, lab1 = 'Model 1', lab2 = 'Model 2'):
    plt.clf()   
    
    plt.rcParams['figure.figsize'] = (25.0, 5.0) 
    epochs = range(len(loss1))
    
    plt.plot(epochs, loss1, 'bo', label='Training loss for ' + lab1)
    plt.plot(epochs, val_loss1, 'b', label='Validation loss for ' + lab1)
    plt.plot(epochs, loss2, 'ro', label='Training loss for ' + lab2)
    plt.plot(epochs, val_loss2, 'r', label='Validation loss for ' + lab2)
    plt.title('Comparison of Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [10]:

models_dir = '/kaggle/working/models/'
if not os.path.exists(models_dir):
    os.mkdir(models_dir)


original_dataset_dir = '/kaggle/working/train/'

base_dir = '/kaggle/working/cats_and_dogs_8000/'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
test_dir = os.path.join(base_dir, 'test1')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
if not os.path.exists(train_cats_dir):
    os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
if not os.path.exists(train_dogs_dir):
    os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
if not os.path.exists(validation_cats_dir):
    os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
if not os.path.exists(validation_dogs_dir):
    os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
if not os.path.exists(test_cats_dir):
    os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
if not os.path.exists(test_dogs_dir):
    os.mkdir(test_dogs_dir)

fnames = ['cat.{}.jpg'.format(i) for i in range(4000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(4000, 6000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(6000, 8000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(4000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(4000, 6000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(6000, 8000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [11]:
model8k = models.Sequential()
model8k.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_rows, img_cols, 3)))
model8k.add(layers.BatchNormalization())
model8k.add(layers.MaxPooling2D((2, 2)))
model8k.add(layers.Dropout(0.1))

model8k.add(layers.Conv2D(64, (3, 3), activation='relu'))
model8k.add(layers.BatchNormalization())
model8k.add(layers.MaxPooling2D((2, 2)))
model8k.add(layers.Dropout(0.15))

model8k.add(layers.Conv2D(128, (3, 3), activation='relu'))
model8k.add(layers.BatchNormalization())
model8k.add(layers.MaxPooling2D((2, 2)))
model8k.add(layers.Dropout(0.20))

model8k.add(layers.Conv2D(128, (3, 3), activation='relu'))
model8k.add(layers.BatchNormalization())
model8k.add(layers.MaxPooling2D((2, 2)))
model8k.add(layers.Dropout(0.25))

model8k.add(layers.Conv2D(256, (3, 3), activation='relu'))
model8k.add(layers.BatchNormalization())
model8k.add(layers.MaxPooling2D((2, 2)))
model8k.add(layers.Flatten())
model8k.add(layers.Dropout(0.5))

model8k.add(layers.Dense(512, activation='relu'))
model8k.add(layers.Dense(1, activation='sigmoid'))

model8k.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-3),
              metrics=['acc'])
model8k.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0

In [12]:

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(img_rows, img_cols),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_rows, img_cols),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [ ]:
history8k = model8k.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      callbacks = learning_rate_reduction)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
250/250 [==============================] - 69s 258ms/step - loss: 1.2841 - acc: 0.5456 - val_loss: 0.9169 - val_acc: 0.5025
Epoch 2/100
250/250 [==============================] - 65s 261ms/step - loss: 0.6394 - acc: 0.6447 - val_loss: 0.7988 - val_acc: 0.5590
Epoch 3/100
103/250 [===========>..................] - ETA: 32s - loss: 0.5612 - acc: 0.7240

In [ ]:
model8k.save(models_dir + 'cats_and_dogs_8k_2.h5')

In [ ]:
acc8k = history8k.history['acc']
val_acc8k = history8k.history['val_acc']
loss8k = history8k.history['loss']
val_loss8k = history8k.history['val_loss']

print(val_acc8k)

PlotAccuracyComparison(acc8k, val_acc8k, lab = 'Model 8k')
PlotLossComparison(loss8k, val_loss8k, lab = 'Model 8k')